In [1]:
%%writefile cudamatrixmultiply.cu
#include <stdio.h>

__global__ void matMul(int *A, int *B, int *C, int N) {
    int row = threadIdx.y;
    int col = threadIdx.x;
    int sum = 0;

    for (int k = 0; k < N; k++)
        sum += A[row * N + k] * B[k * N + col];

    C[row * N + col] = sum;
}

int main() {
    int N = 2;
    int A[4] = {1, 2, 3, 4};
    int B[4] = {5, 6, 7, 8};
    int C[4];

    int *d_A, *d_B, *d_C;

    cudaMalloc(&d_A, 4 * sizeof(int));
    cudaMalloc(&d_B, 4 * sizeof(int));
    cudaMalloc(&d_C, 4 * sizeof(int));

    cudaMemcpy(d_A, A, 4 * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, 4 * sizeof(int), cudaMemcpyHostToDevice);

    dim3 threads(2, 2);
    matMul<<<1, threads>>>(d_A, d_B, d_C, N);

    cudaMemcpy(C, d_C, 4 * sizeof(int), cudaMemcpyDeviceToHost);

    for (int i = 0; i < 4; i++)
        printf("%d ", C[i]);

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Writing cudamatrixmultiply.cu


In [2]:
!nvcc cudamatrixmultiply.cu -o cudamatrixmultiply -arch=sm_70

In [3]:
!./cudamatrixmultiply

19 22 43 50 